# Reader Monad

## Outline

* Incentive for the Reader monad

* Definition of the Reader monad

* Reader monad examples

In this lesson, we will learn about the Reader monad type and how you can use it.

## Incentive for Reader monad

Imagine we have a global state which is represented by a variable. An example would be that we read 2 variables from the operating system environment and store them into a variable of type `Environment`. We define the function `loadEnv` where we try to read the values from the variables `PATH` which is defined for UNIX type OS and the variable `MYPARAM` which is probably not defined for any OS type. 

In [ ]:
import Data.Maybe (fromMaybe)
import System.Environment (lookupEnv)

data Environment = Environment
  { param1 :: String
  , param2 :: String
  }

loadEnv :: IO Environment
loadEnv = do
  p1 <- lookupEnv "PATH"
  p2 <- lookupEnv "MYPARAM"
  return $ Environment
    (fromMaybe "" p1)
    (fromMaybe "" p2)

Now if we want to use this environment in some functions that process it, we have to pass the data around with the functions. Let's create a function that calculates the length of the variables and then sums it and another function that prints the result of the sum. Here is how the code looks like.

In [ ]:
func1 :: Environment -> String
func1 env = "Result: " ++ (show (func2 env))

func2 :: Environment -> Int
func2 env = l1 + l2
  where
    l1 = length (param1 env)
    l2 = length (param2 env)

We can now write the main execution of our program where we read in the environment and print the result of processing the data with our two functions.

In [ ]:
main1 :: IO ()
main1 = do
  env <- loadEnv
  let str = func1 env
  print str

main1

This is a very simple example, but as complexity of programs increases you have to pass around a lot of data. For this reason the Reader monad was invented which stores the state in its environment variable and can simply pass it around between different Reader monads that process the data.

## Definition of the Reader monad

You can find the definition of the Reader monad on Hoogle. The definition shown here uses multi-parameter type classes and funDeps, which are not standard Haskell 2010. It is still useful to look at it but it is not necessary to fully understand these details to make use of the Reader monad. 

In [ ]:
newtype Reader e a = Reader { runReader :: (e -> a) }

instance Monad (Reader e) where
    return a         = Reader $ \e -> a
    (Reader r) >>= f = Reader $ \e -> runReader (f (r e)) e

A function that can retrieve the environment variable is called `ask`. It is included in the module `Control.Monad.Reader`. Another function called `runReader` can retrieve the final value of the Reader monad and is also included in the same module. Here is a simple example how to define a reader monad and use the `ask` and `runReader` functions.

In [ ]:
import Control.Monad.Reader

age :: Reader Int String
age = do
    env <- ask
    return ("My age is: " ++ (show env))

ageDouble :: Reader Int String
ageDouble = do
    env <- ask
    return ("Two times my age is: " ++ (show $ 2*env))

bothAges :: Reader Int String
bothAges = do
    a <- age
    ad <- ageDouble
    return (a ++ "\n" ++ ad)

main2 :: IO ()
main2 = do
    myAge <- read <$> getLine
    print $ (runReader bothAges) myAge

main2

You see that when calling the `age` and `ageDouble` Reader monads inside the `bothAges` Reader monad the environment of the bothAges Reader monad which is in our case of type Int, is passed onto them. And you can then process this data accordingly. 

## Reader monad examples

Let's implement the example from the first chapter with the use of the Reader monad.

In [ ]:
func1' :: Reader Environment String
func1' = do
  res <- func2'
  return ("Result: " ++ show res)

func2' :: Reader Environment Int
func2' = do
  env <- ask
  let res = func2 env
  return res

main3 :: IO ()
main3 = do
  env <- loadEnv
  let str = runReader func1' env
  print str

main3

We could define the functionality of `func2` also inside `func2'` but to make it simpler we just make a function call to the existing function.

Another more complicated example is how to lookup variables if you have them stored in a list. The list contains tuples where the first part is the variable name and the second is variable either of type string or of type `Template`, which we will define for this example. What we want to show here is that it is also possible to pass in data to your Reader monad beside the intial environment. 

In [ ]:
data Template = S String | V Template | T Template deriving Show

data Environment = Env {templates::[(String,Template)],
                        variables::[(String,String)]}

-- lookup a variable from the environment
lookupVar :: String -> Environment -> Maybe String
lookupVar name env = lookup name (variables env)

-- lookup a template from the environment
lookupTemplate :: String -> Environment -> Maybe Template
lookupTemplate name env = lookup name (templates env)

-- resolve a template into a string
resolve :: Template -> Reader Environment (String)
resolve (S s)    = return s
resolve (V t)    = do varName  <- resolve t
                      varValue <- asks (lookupVar varName)
                      return $ maybe "" id varValue
resolve (T t)    = do tmplName <- resolve t
                      body     <- asks (lookupTemplate tmplName)
                      return $ maybe "" show body

template1 :: Template
template1 = V (S "varName1")

template2 :: Template
template2 = T (S "varName2")

myEnv :: Environment
myEnv = Env {
      templates = [("varName1",template1),("varName2",template2)],
      variables = [("varName1","value1"),("varName2","value2")]
    }

main :: IO ()
main = do
    print $ runReader (resolve template1) myEnv
    print $ runReader (resolve template2) myEnv

main

Below you have also the definition of the `asks` function that uses the functionality of the `MonadReader` type class.

```haskell
class Monad m => MonadReader r m | m -> r where
    {-# MINIMAL (ask | reader), local #-}
    -- | Retrieves the monad environment.
    ask   :: m r
    ask = reader id

    -- | Executes a computation in a modified environment.
    local :: (r -> r) -- ^ The function to modify the environment.
          -> m a      -- ^ @Reader@ to run in the modified environment.
          -> m a

    -- | Retrieves a function of the current environment.
    reader :: (r -> a) -- ^ The selector function to apply to the environment.
           -> m a
    reader f = do
      r <- ask
      return (f r)

-- | Retrieves a function of the current environment.
asks :: MonadReader r m
    => (r -> a) -- ^ The selector function to apply to the environment.
    -> m a
asks = reader
```

## Recap

In this lesson we've discussed:

- the motivation for introducing the Reader monad type 

- the definition of the Reader monad type and a simple example

- examples that use the Reader monad type to solve a problem